In [1]:
train_len = 295246830
one_len = 1589906
zero_len = 293656924
protein_map = {'BRD4': 1, 'HSA': 2, 'sEH': 3}
vocab = {'C': 6825082866, '#': 81527490, '@': 511451694, 'H': 456489972, '=': 1406606874, 'O': 2554179786,
         'N': 2469595230, 'c': 12257477022, '-': 438483636, '.': 216945504, 'l': 491088828, 'B': 123330132,
         'r': 121915914, 'n': 1997759694, 'D': 295246830, 'y': 295246830, 'o': 67918650, 's': 156618468,
         'S': 90662574, 'F': 492710238, '+': 65206260, 'i': 1414026, '/': 11547096, 'I': 23972994}

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import LongType, IntegerType, StructType, StructField, ArrayType, DoubleType, StringType
from pyspark.ml.linalg import SparseVector

from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, StringIndexerModel, OneHotEncoderModel
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

import os
import pandas as pd
import numpy as np
import joblib

# from xgboost.spark import SparkXGBClassifier
# from functools import wraps
# import xgboost as xgb

# from sklearn.metrics import classification_report, roc_auc_score, average_precision_score

from concurrent.futures import ThreadPoolExecutor
from joblib import Parallel, delayed
# from rdkit import Chem
# from rdkit.Chem import Descriptors, Lipinski, rdmolops, AllChem, rdchem, rdEHTTools, rdMolDescriptors
# from tqdm.auto import tqdm
from tqdm import tqdm
# from padelpy import from_smiles
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
# # for 256 Gb and 64 Cores
# spark = (
#     SparkSession
#     .builder
#     .appName("leash belka3")
#     .config("spark.driver.memory", "48g")  # Increased driver memory
#     .config("spark.executor.memory", "48g")  # Increased executor memory
#     .config("spark.executor.instances", "16")  # 16 executors
#     .config("spark.executor.cores", "4")  # 4 cores per executor
#     .config("spark.driver.maxResultSize", "4g")  # Driver result size limit
#     .config("spark.local.dir", "temp")  # Specify a directory with enough space
#     # .config("spark.local.dir", "/scratch/23m1521/temp")  # Specify a directory with enough space
#     .config("spark.shuffle.file.buffer", "128k")  # Shuffle buffer size
#     .config("spark.memory.fraction", "0.8")  # Memory fraction for tasks
#     .config("spark.shuffle.memoryFraction", "0.6")  # Shuffle memory fraction
#     .config("spark.executor.javaOptions", "-Xmx48g")  # JVM heap size for executors
#     .master("local[64]")  # Use all 64 cores on the machine
#     .getOrCreate()
# )

# spark
spark = (
    SparkSession
    .builder
    .appName("leash belka346765")
    .config("spark.driver.memory", "64g")  # Increased driver memory for large jobs
    .config("spark.executor.memory", "64g")  # Increased executor memory
    .config("spark.executor.instances", "32")  # 32 executors
    .config("spark.executor.cores", "2")  # 2 cores per executor
    .config("spark.driver.maxResultSize", "8g")  # Driver result size limit
    .config("spark.local.dir", "temp")  # Ensure high-speed storage
    .config("spark.shuffle.file.buffer", "1024k")  # Larger shuffle buffer for better IO
    .config("spark.memory.fraction", "0.85")  # Increased memory for tasks
    .config("spark.shuffle.memoryFraction", "0.7")  # Increased shuffle memory
    .config("spark.executor.javaOptions", "-Xmx64g")  # JVM heap size for executors
    .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:1.0.8")
    .config("spark.jars.repositories", "https://mmlspark.azureedge.net/maven")
    .master("local[*]")  # Use all 64 cores on the machine
    .getOrCreate()
)
spark

# SparkSession for 128 GB RAM and 64 cores
# spark = (
#     SparkSession
#     .builder
#     .appName("Optimized Spark for 128GB RAM and 64 Cores")
#     .config("spark.driver.memory", "64g")  # 64GB for driver memory
#     .config("spark.executor.memory", "64g")  # 64GB for executor memory
#     .config("spark.executor.instances", "16")  # 16 executors
#     .config("spark.executor.cores", "4")  # 4 cores per executor (total = 64 cores)
#     .config("spark.driver.maxResultSize", "8g")  # Driver result size limit
#     .config("spark.local.dir", "temp")  # Temp directory with enough space
#     .config("spark.shuffle.file.buffer", "512k")  # Increased shuffle buffer size
#     .config("spark.memory.fraction", "0.8")  # Memory fraction for tasks
#     .config("spark.shuffle.memoryFraction", "0.6")  # Shuffle memory fraction
#     .config("spark.executor.javaOptions", "-Xmx64g")  # JVM heap size for executors
#     .master("local[64]")  # Use all 64 cores on the machine
#     .getOrCreate()
# )

# spark

# SynapseML 
# spark = (
#     SparkSession
#     .builder
#     .appName("leash belka3")
#     .config("spark.driver.memory", "48g")  # Increased driver memory
#     .config("spark.executor.memory", "48g")  # Increased executor memory
#     .config("spark.executor.instances", "16")  # 16 executors
#     .config("spark.executor.cores", "4")  # 4 cores per executor
#     .config("spark.driver.maxResultSize", "4g")  # Driver result size limit
#     .config("spark.local.dir", "temp")  # Specify a directory with enough space
#     .config("spark.shuffle.file.buffer", "128k")  # Shuffle buffer size
#     .config("spark.memory.fraction", "0.8")  # Memory fraction for tasks
#     .config("spark.shuffle.memoryFraction", "0.6")  # Shuffle memory fraction
#     .config("spark.executor.javaOptions", "-Xmx48g")  # JVM heap size for executors
#     .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:1.0.8")
#     .config("spark.jars.repositories", "https://mmlspark.azureedge.net/maven")
#     .master("local[64]")  # Use all 64 cores on the machine
#     .getOrCreate()
# )

# spark

# spark = (
#     SparkSession
#     .builder
#     .appName("leash belka3")
#     .config("spark.driver.memory", "64g")  # Increased driver memory
#     .config("spark.executor.memory", "64g")  # Increased executor memory
#     .config("spark.executor.instances", "8")  # Reduced number of executors
#     .config("spark.executor.cores", "8")  # Increased cores per executor
#     .config("spark.driver.maxResultSize", "4g")  # Driver result size limit
#     .config("spark.local.dir", "temp")  # Specify a directory with enough space
#     .config("spark.shuffle.file.buffer", "128k")  # Shuffle buffer size
#     .config("spark.memory.fraction", "0.8")  # Memory fraction for tasks
#     .config("spark.shuffle.memoryFraction", "0.7")  # Shuffle memory fraction
#     .config("spark.executor.javaOptions", "-Xmx64g")  # JVM heap size for executors
#     .config("spark.sql.shuffle.partitions", "1000")  # Increase shuffle partitions
#     .config("spark.ui.enabled", "true")  # Enable Spark UI
#     .master("local[8]")  # Reduced number of cores for local mode
#     .getOrCreate()
# )

# spark


25/01/03 03:35:32 WARN Utils: Your hostname, kanjur resolves to a loopback address: 127.0.1.1; using 10.119.2.14 instead (on interface eno3)
25/01/03 03:35:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
https://mmlspark.azureedge.net/maven added as a remote repository with the name: repo-1


:: loading settings :: url = jar:file:/home/23m1521/.conda/envs/cuda_env2/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/23m1521/.ivy2/cache
The jars for the packages stored in: /home/23m1521/.ivy2/jars
com.microsoft.azure#synapseml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-798cb5a4-399c-4508-ae50-3d276eff8f12;1.0
	confs: [default]
	found com.microsoft.azure#synapseml_2.12;1.0.8 in spark-list
	found com.microsoft.azure#synapseml-core_2.12;1.0.8 in central
	found org.apache.spark#spark-avro_2.12;3.4.1 in central
	found org.tukaani#xz;1.9 in central
	found commons-lang#commons-lang;2.6 in central
	found org.scalactic#scalactic_2.12;3.2.14 in central
	found org.scala-lang#scala-reflect;2.12.15 in central
	found io.spray#spray-json_2.12;1.3.5 in central
	found com.jcraft#jsch;0.1.54 in central
	found org.apache.httpcomponents.client5#httpclient5;5.1.3 in central
	found org.apache.httpcomponents.core5#httpcore5;5.1.3 in central
	found org.apache.httpcomponents.core5#httpcore5-h2;5.1.3 in central
	found org.slf4j#slf4j-api;1.7.2

In [3]:
datadir = "/home/23m1521/ashish/kaggle/full_feat_tok_df_vectors.parquet"
chunks_path = sorted([os.path.join(datadir, i) for i in os.listdir(datadir) if i.endswith(".parquet")])
total_chunks = len(chunks_path)
print(total_chunks)

200


In [4]:
full_feat_tok_df_vectors = spark.read.format('parquet').load(chunks_path[1])
print(full_feat_tok_df_vectors.count())
full_feat_tok_df_vectors.show()

1476728
+----+--------------------+-----+
|  id|             vectors|binds|
+----+--------------------+-----+
|  65|(190,[0,1,2,3,4,5...|    0|
| 191|(190,[0,1,2,3,4,5...|    0|
| 418|(190,[0,1,2,3,4,5...|    0|
| 541|(190,[0,1,2,3,4,5...|    0|
| 558|(190,[0,1,2,3,4,5...|    0|
|1010|(190,[0,1,2,3,4,5...|    0|
|1224|(190,[0,1,2,3,4,5...|    0|
|1258|(190,[0,1,2,3,4,5...|    0|
|1277|(190,[0,1,2,3,4,5...|    0|
|1360|(190,[0,1,2,3,4,5...|    0|
|1840|(190,[0,1,2,3,4,5...|    0|
|2173|(190,[0,1,2,3,4,5...|    0|
|2812|(190,[0,1,2,3,4,5...|    0|
|2895|(190,[0,1,2,3,4,5...|    0|
|2906|(190,[0,1,2,3,4,5...|    0|
|2941|(190,[0,1,2,3,4,5...|    0|
|3009|(190,[0,1,2,3,4,5...|    0|
|3015|(190,[0,1,2,3,4,5...|    0|
|3061|(190,[0,1,2,3,4,5...|    0|
|3106|(190,[0,1,2,3,4,5...|    0|
+----+--------------------+-----+
only showing top 20 rows



In [5]:
class_counts = dict(full_feat_tok_df_vectors.groupBy("binds").count().collect())
contamination = class_counts[1]/class_counts[0]
class_counts, contamination

({0: 1468867, 1: 7861}, 0.00535174389512461)

## Train Isolation Forest model

In [9]:
def make_dataset2(df, test=False):
    df = df.toPandas()
    df.vectors = df.vectors.map(lambda x: x.toArray())
    if test == True:
        return df.id.values, np.array([i for i in df.vectors.values]), None, None
    return df.id.values, np.array([i for i in df.vectors.values]), df.binds.values

ids, X, y = make_dataset2(full_feat_tok_df_vectors)

In [19]:
from sklearn.ensemble import IsolationForest
import numpy as np
import pandas as pd
from sklearn.metrics import auc, precision_recall_curve
from sklearn.preprocessing import LabelBinarizer

isolation_forest = IsolationForest(
    n_estimators=500,
    max_samples=0.5,  # Or "auto" or a specific integer
    contamination=contamination,  # Or "auto" for default behavior
    max_features=0.7,  # Or an integer for a fixed number of features
    bootstrap=True,  # Whether to use bootstrapping
    n_jobs=-1,      # Use all available processors
    random_state=42,  # For reproducibility
    verbose=0,       # Verbosity level
    warm_start=False # Reuse the solution of the previous call to fit and add more estimators to the ensemble, otherwise, just fit a whole new forest.
)

isolation_forest.fit(X)

IsolationForest(bootstrap=True, contamination=0.00535174389512461,
                max_features=0.7, max_samples=0.5, n_estimators=500, n_jobs=-1,
                random_state=42)

In [20]:
predictions = isolation_forest.predict(X) # 1 for inliers, -1 for outliers
scores = isolation_forest.decision_function(X) # Anomaly score for each sample
outlier_indices = np.where(predictions == -1)[0]
print("Outlier indices:", outlier_indices)
print(scores)

Outlier indices: [    231    1523    1546 ... 1476708 1476709 1476712]
[0.08211198 0.07574384 0.0801966  ... 0.02335361 0.05317712 0.03663002]


In [35]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import CalibratedClassifierCV

def anomaly_scores_to_probabilities_calibration(y_true, scores, higher_is_anomalous=True, method='isotonic'):
    """Calibrates anomaly scores to probabilities using isotonic regression or Platt scaling."""
    if higher_is_anomalous:
        scores = -scores #Isotonic regression work with increasing values
    if method == 'isotonic':
        ir = IsotonicRegression()
        calibrated_scores = ir.fit_transform(scores, y_true)
    elif method == 'platt':
        calibrated_scores = CalibratedClassifierCV(
            estimator = LogisticRegression(n_jobs=-1),
            cv=5,
            n_jobs=-1
            ).fit(scores.reshape(-1, 1), y_true).predict_proba(scores.reshape(-1,1))[:,1]
    else:
        raise ValueError("Method must be 'isotonic' or 'platt'")
    if higher_is_anomalous:
        probabilities_1 = calibrated_scores
        probabilities_0 = 1- calibrated_scores
    else:
        probabilities_0 = calibrated_scores
        probabilities_1 = 1- calibrated_scores
    return np.column_stack((probabilities_0, probabilities_1))


probabilities_normal = anomaly_scores_to_probabilities_calibration(y, scores, 
                                                                   higher_is_anomalous=False, method='platt'
                                                                   )
print(probabilities_normal)

[[0.00505229 0.99494771]
 [0.00484956 0.99515044]
 [0.00498959 0.99501041]
 ...
 [0.00367181 0.99632819]
 [0.00424839 0.99575161]
 [0.00390274 0.99609726]]


In [36]:
probabilities_normal[:,1]

array([0.99494771, 0.99515044, 0.99501041, ..., 0.99632819, 0.99575161,
       0.99609726])

In [ ]:
def calculate_auc_pr(y_true, y_scores):
    try:
        # Handle multiclass case using one-vs-rest binarization
        if len(np.unique(y_true)) > 2:
            lb = LabelBinarizer()
            y_true_bin = lb.fit_transform(y_true)
            if y_true_bin.shape[1] == 1:
                y_true_bin = np.hstack([1 - y_true_bin, y_true_bin]) #handle the case where labelbinarizer return only one column
            auc_pr_scores = []
            for i in range(y_true_bin.shape[1]):
                precision, recall, _ = precision_recall_curve(y_true_bin[:, i], y_scores[:, i])
                auc_pr = auc(recall, precision)
                auc_pr_scores.append(auc_pr)
            return np.mean(auc_pr_scores) # Return the mean AUC-PR over all classes.
        else: #binary case
            precision, recall, _ = precision_recall_curve(y_true, y_scores)
            return auc(recall, precision)
    except ValueError as e:
        print(f"Error calculating AUC-PR: {e}")
        return None  # Handle cases like only one class being present



auc_pr_binary = calculate_auc_pr(y, scores)
print(f"Binary AUC-PR: {auc_pr_binary}")
calculate_auc_pr(y, probabilities_normal[:,1]), calculate_auc_pr(y, probabilities_normal[:,0])

Binary AUC-PR: 0.005970386939440327


(0.00466528435253541, 0.005970386939440327)

: 

In [6]:
from synapse.ml.isolationforest import IsolationForest

In [7]:
isolationForest = IsolationForest(
    bootstrap=False,
    contamination=contamination,
    contaminationError=0.01 * contamination,
    featuresCol="vectors",
    maxFeatures=1.0,
    maxSamples=0.2,
    numEstimators=1000,
    predictionCol="predictedLabel",
    randomSeed=42,
    scoreCol="outlierScore"
)

In [ ]:
model = isolationForest.fit(full_feat_tok_df_vectors)

## Perform inferencing

In [9]:
full_feat_tok_df_vectors = model.transform(full_feat_tok_df_vectors)

In [ ]:
full_feat_tok_df_vectors.show()

25/01/02 23:47:30 WARN DAGScheduler: Broadcasting large task binary with size 81.0 MiB
25/01/02 23:47:33 WARN DAGScheduler: Broadcasting large task binary with size 81.0 MiB
25/01/02 23:47:37 WARN DAGScheduler: Broadcasting large task binary with size 81.0 MiB


+----+--------------------+-----+-------------------+--------------+
|  id|             vectors|binds|       outlierScore|predictedLabel|
+----+--------------------+-----+-------------------+--------------+
|  65|(190,[0,1,2,3,4,5...|    0|0.38952457174064203|           0.0|
| 191|(190,[0,1,2,3,4,5...|    0| 0.3935112108060045|           0.0|
| 418|(190,[0,1,2,3,4,5...|    0|0.40572750302732014|           0.0|
| 541|(190,[0,1,2,3,4,5...|    0|0.39243424010460193|           0.0|
| 558|(190,[0,1,2,3,4,5...|    0| 0.3927223271193177|           0.0|
|1010|(190,[0,1,2,3,4,5...|    0| 0.3851334912164859|           0.0|
|1224|(190,[0,1,2,3,4,5...|    0| 0.4213978244924502|           0.0|
|1258|(190,[0,1,2,3,4,5...|    0|0.40407570713910124|           0.0|
|1277|(190,[0,1,2,3,4,5...|    0|0.42491467645957937|           0.0|
|1360|(190,[0,1,2,3,4,5...|    0| 0.4302462718482658|           0.0|
|1840|(190,[0,1,2,3,4,5...|    0|0.39414203209719023|           0.0|
|2173|(190,[0,1,2,3,4,5...|    0|0

In [ ]:
full_feat_tok_df_vectors.groupBy("predictedLabel").count().collect()

25/01/02 23:47:51 WARN DAGScheduler: Broadcasting large task binary with size 81.0 MiB
25/01/02 23:51:09 WARN DAGScheduler: Broadcasting large task binary with size 81.0 MiB


[Row(predictedLabel=0.0, count=1468796), Row(predictedLabel=1.0, count=7932)]

In [ ]:
pr_auc_evaluator = BinaryClassificationEvaluator(labelCol="binds", 
                                                 rawPredictionCol="outlierScore", 
                                                 metricName="areaUnderPR")
pr_auc = pr_auc_evaluator.evaluate(full_feat_tok_df_vectors)
print(f"PR AUC (approx MAP): {pr_auc}")

# pr_auc = pr_auc_evaluator.evaluate(full_predictions)
# print(f"PR AUC (approx MAP): {pr_auc}")

25/01/02 23:51:15 WARN DAGScheduler: Broadcasting large task binary with size 81.0 MiB


PR AUC (approx MAP): 0.004571768784415767


In [14]:
pr_auc_evaluator = BinaryClassificationEvaluator(labelCol="binds", 
                                                 rawPredictionCol="predictedLabel", 
                                                 metricName="areaUnderPR")
pr_auc = pr_auc_evaluator.evaluate(full_feat_tok_df_vectors)
print(f"PR AUC (approx MAP): {pr_auc}")

# pr_auc = pr_auc_evaluator.evaluate(full_predictions)
# print(f"PR AUC (approx MAP): {pr_auc}")

25/01/02 23:57:11 WARN DAGScheduler: Broadcasting large task binary with size 81.0 MiB


PR AUC (approx MAP): 0.00436029818974985


In [13]:
full_feat_tok_df_vectors.where("binds == 1").show()

25/01/02 23:56:15 WARN DAGScheduler: Broadcasting large task binary with size 81.0 MiB
25/01/02 23:56:18 WARN DAGScheduler: Broadcasting large task binary with size 81.0 MiB
25/01/02 23:56:22 WARN DAGScheduler: Broadcasting large task binary with size 81.0 MiB


+-------+--------------------+-----+-------------------+--------------+
|     id|             vectors|binds|       outlierScore|predictedLabel|
+-------+--------------------+-----+-------------------+--------------+
|  43442|(190,[0,1,2,3,4,5...|    1|0.42633382085345667|           0.0|
|  69784|(190,[0,1,2,3,4,5...|    1|0.39403948946242146|           0.0|
| 465211|(190,[0,1,2,3,4,5...|    1| 0.3849148945590878|           0.0|
| 498101|(190,[0,1,2,3,4,5...|    1| 0.3895195185317123|           0.0|
| 509867|(190,[0,1,2,3,4,5...|    1| 0.3785149765255084|           0.0|
| 510692|(190,[0,1,2,3,4,5...|    1|0.38751757206339965|           0.0|
| 510851|(190,[0,1,2,3,4,5...|    1| 0.3787319101572828|           0.0|
| 532509|(190,[0,1,2,3,4,5...|    1| 0.3971351162532173|           0.0|
| 797769|(190,[0,1,2,3,4,5...|    1|0.37282339852420276|           0.0|
| 930648|(190,[0,1,2,3,4,5...|    1| 0.4288135725238536|           0.0|
| 967915|(190,[0,1,2,3,4,5...|    1| 0.4121725667361934|        

///////////////////////////////////////////////////////////////////////////////////////////

In [10]:
def load_df_chunk(path):
    return spark.read.format('parquet').load(path)

def add_sample_weights(df):
    class_counts = df.groupBy("binds").count().collect()
    total_count = sum(row["count"] for row in class_counts)
    class_weights = {row["binds"]: total_count / (2 * row["count"]) for row in class_counts}
    return class_counts, df.withColumn("sample_weights", when(col("binds") == 0, class_weights[0]).when(col("binds") == 1, class_weights[1]))

def make_dataset(df, chunk_df_count):
    def process_row(row):
        return (row['vectors'].toArray(), row['binds'], row['sample_weights'])
    features, labels, weights = [], [], []
    for feature, label, weight in tqdm(df.rdd.map(process_row).toLocalIterator(), total=chunk_df_count):
        features.append(feature)
        labels.append(label)
        weights.append(weight)
    return features, labels, weights

def make_dataset2(df, test=False):
    df = df.toPandas()
    df.vectors = df.vectors.map(lambda x: x.toArray())
    if test == True:
        return df.id.values, np.array([i for i in df.vectors.values]), None, None
    return df.id.values, np.array([i for i in df.vectors.values]), df.binds.values, df.sample_weights.values

def save_checkpoint(model, params, i, evals_result,  path, save_name):
    os.makedirs(path, exist_ok=True)
    model.save_model(os.path.join(path, f"{save_name}.json"))
    joblib.dump({"params": params, 'i': i, "evals_result": evals_result}, os.path.join(path, f"{save_name}_params.joblib"))
    print("Model saved at", path)

def load_checkpoint(path, save_name):
    model = xgb.Booster()
    model.load_model(os.path.join(path, f"{save_name}.json"))
    ckpt = joblib.load(os.path.join(path, f"{save_name}_params.joblib"))
    params, i = ckpt['params'], ckpt['i']
    print("Model loaded from", path)
    return model, params, i

def train_IsolationForest(X, xgb_model=None):
    model = IsolationForest(
        n_estimators=100,
        contamination=contamination,
        n_jobs=-1,
        random_state=42,
        warm_start=True
        ).fit(X)
    return bst, evals_result, best_params1

def delete_df_chunk(df):
    df.unpersist()
    del df

def spark_suppress_logs(level="ERROR", reset_level="INFO"):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            spark.sparkContext.setLogLevel(level)
            try:
                return func(*args, **kwargs)
            finally:
                spark.sparkContext.setLogLevel(reset_level)
        return wrapper
    return decorator


@spark_suppress_logs()
def incrementally_train():
    with tqdm(total=total_chunks, dynamic_ncols=True) as pbar1:
        sample_count = 0
        xgb_model = None
        ckpt_dir = "Incrementally_train_XGB_ckpt"
        
        for i, chunk_path in enumerate(chunks_path):
            # if i > 1:
            #     break
        
            pbar1.set_description(f"Chunk: {i+1}")
            
            
            # --- Load chunk --------------------------------------
            chunk_df = load_df_chunk(chunk_path)
            chunk_df = add_sample_weights(chunk_df)
            chunk_df = chunk_df.repartition(1)
            chunk_df_count = chunk_df.count()
            
            # --- Getting Dataset ---------------------------------
            _, features, labels, weights = make_dataset2(chunk_df)
            dtrain = xgb.DMatrix(data=features, label=labels, weight=weights, nthread=25)
            
            # # --- Train ------------------------------------------
            if xgb_model is None:
                xgb_model, evals_result, params = train_xgb(dtrain)
            else:
                xgb_model, evals_result, params = train_xgb(dtrain, xgb_model)
            save_checkpoint(xgb_model, params, i, evals_result, ckpt_dir, f"_{i+1}_ckpt")
            
            # # --- Model Evaluation -------------------------------
            loss = np.mean(evals_result['train']['logloss'])
            aucpr = np.mean(evals_result['train']['aucpr'])
            print(f"Chunk {i+1} trained. Loss: {loss}, AUCPR: {aucpr}")
            
            
            # --- Clean up ----------------------------------------
            # delete_df_chunk(chunk_df)
            del chunk_df, features, labels, weights, dtrain, _
            sample_used = sample_count + chunk_df_count
            sample_used_percentage = (sample_used / train_len) * 100
            remaining_samples = train_len - sample_used
            remaining_samples_percentage = (remaining_samples / train_len) * 100
            pbar1.set_postfix_str(
                f"{sample_used} ({sample_used_percentage:.2f}%) samples used," 
                f"{remaining_samples} ({remaining_samples_percentage:.2f}%) samples remaining"
            )
            pbar1.update(1)
            sample_count += chunk_df_count
            
        # --- Save final model -----------------------------------
        save_checkpoint(xgb_model, params, i, evals_result, ckpt_dir, f"Final_ckpt")


@spark_suppress_logs()
def incrementally_inference(model, chunks_path, save_dir):
    with tqdm(total=total_chunks, dynamic_ncols=True) as pbar1:
        sample_count = 0
        os.makedirs(save_dir, exist_ok=True)
        # id_array = np.array([], dtype=np.uint64)
        # preds_array = np.array([], dtype=np.float128)
        
        for i, chunk_path in enumerate(chunks_path):
            pbar1.set_description(f"Chunk: {i+1}")
            
            
            # --- Load chunk --------------------------------------
            chunk_df = load_df_chunk(chunk_path)
            # chunk_df = add_sample_weights(chunk_df)
            chunk_df = chunk_df.repartition(1)
            chunk_df_count = chunk_df.count()
            
            # --- Getting Dataset ---------------------------------
            ids, features, labels, weights = make_dataset2(chunk_df, test=True)
            dtest = xgb.DMatrix(data=features, nthread=25)
            
            # # --- Predict ---------------------------------------
            preds = model.predict(dtest)
    
            # preds_array = np.concatenate([preds_array, preds])
            # id_array = np.concatenate([id_array, ids])
            
            # --- Save predictions -------------------------------
            chunk_save_path = os.path.join(save_dir, f"chunk_{i + 1}.npz")
            np.savez(chunk_save_path, ids=ids, preds=preds)
    
            
            # --- Clean up ----------------------------------------
            del chunk_df, features, labels, weights, dtest, ids, preds
            sample_used = sample_count + chunk_df_count
            sample_used_percentage = (sample_used / train_len) * 100
            remaining_samples = train_len - sample_used
            remaining_samples_percentage = (remaining_samples / train_len) * 100
            pbar1.set_postfix_str(
                f"{sample_used} ({sample_used_percentage:.2f}%) samples used," 
                f"{remaining_samples} ({remaining_samples_percentage:.2f}%) samples remaining"
            )
            pbar1.update(1)
            sample_count += chunk_df_count

    return {'id': id_array, 'binds': preds_array}

In [5]:
# spark.sparkContext.setLogLevel("ERROR")
# incrementally_train()
# spark.sparkContext.setLogLevel("INFO")

## Model Evaluation

In [6]:
# full_feat_tok_df_vectors = spark.read.format('parquet').load(chunks_path[1])
# print(full_feat_tok_df_vectors.count())
# full_feat_tok_df_vectors.show()

In [7]:
params = joblib.load("Incrementally_train_XGB_ckpt/Final_ckpt_params.joblib")['params']
model = xgb.Booster(params=params)
model.load_model("Incrementally_train_XGB_ckpt/Final_ckpt.json")

In [ ]:
full_preds = incrementally_inference(model, chunks_path, save_dir="Incrementally_Inference3_Preds")

/home/23m1521/.conda/envs/cuda_env2/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:38:52] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1733179637554/work/src/learner.cc:740: 
Parameters: { "rate_drop", "skip_drop" } are not used.

  warnings.warn(smsg, UserWarning)


In [54]:
chunk_pred_df = pd.DataFrame(chunk_preds)
chunk_pred_df

,id,binds
0,65.0,0.152128
1,191.0,0.017978
2,418.0,0.010289
3,541.0,0.069637
4,558.0,0.064629
...,...,...
1476723,295246296.0,0.370344
1476724,295246331.0,0.045317
1476725,295246454.0,0.609166
1476726,295246584.0,0.015393


In [57]:
chunk_pred_df['id'] = chunk_pred_df['id'].astype(np.int64)
chunk_pred_df = chunk_pred_df.sort_values(by=['id'])
chunk_pred_df

,id,binds
0,65,0.152128
1,191,0.017978
2,418,0.010289
3,541,0.069637
4,558,0.064629
...,...,...
1476723,295246296,0.370344
1476724,295246331,0.045317
1476725,295246454,0.609166
1476726,295246584,0.015393
